# Crawler for Even dictionary

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import json
from html2text import html2text
from tqdm.auto import tqdm
from string import punctuation as punc

Online-version is here

In [2]:
base_link = 'http://jazykirossii.ru/eve.html?page='

Collect all the entries

In [ ]:
new_words = []
pages = range(31, 354)
for i in tqdm(pages):
    page = str(i)
    link = base_link + page
    html = requests.get(link).text
    # word entries have no separate tags
    # and are located between delimiters
    words = re.findall(
        r'(?<=<div class="delim"></div>)(.*?)(?=<div class="delim"></div>)',
        html)
    # for the last word the second delimiter does not appear
    last_word = re.findall(
        r'.*<div class="delim"></div>(.*?)\n<style>',
        html)
    words += last_word
    for word in words:
        # protect and replace
        word = word.replace('[', '\[').replace(']', '\]')
        word = re.sub('<.?br.*?>', '', word)
        word = re.sub('<(.?)b.*?>', '[\g<1>b]', word)
        # add some colours!
        if '<i style="color:green">' in word:
            indices = [i.end() for i in re.finditer('<i style="color:green">', word)]
            for index in indices:
                word = word[:index] + word[index:].replace('</i>', '[/c][/i]')
            word = word.replace('<i style="color:green">', '[i][c green]')
        word = re.sub('<(.?)i.*?>', '[\g<1>i] ', word)
        # render diacritics and special symbols
        word = html2text(word)
        new_words.append(re.sub('\s+', ' ', word))

  0%|          | 0/323 [00:00<?, ?it/s]

In [ ]:
for_dict = [
    '#NAME "Эвенско-русский словарь (Evn-Rus)"',
    '#INDEX_LANGUAGE\t"Even"',
    '#CONTENTS_LANGUAGE\t"Russian"',
]
dictionary = {}
for word in new_words:
    new_word = re.search('\[b\](.*?)\[/b\]', word)
    for_dict.append(f'{new_word.groups(1)[0]}\n\t{word}')
    dictionary[new_word.groups(1)[0].strip(punc)] = re.sub('\[.+?\]', '', word[new_word.end():]).strip()

In [ ]:
with open('Even.dsl', 'w', encoding='utf-16') as f:
    f.write('\n'.join(for_dict))

In [ ]:
with open('Even.json', 'w', encoding='utf-8') as f:
    json.dump(dictionary, f, ensure_ascii=False, indent=4)